In [6]:
cd ~/mydata/item_profilt_classification/

/mnt/raid1/home/zhuyuhe/mydata/item_profilt_classification


In [2]:
import jieba
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import csv

In [3]:
from tqdm import tqdm
import re

In [4]:
#辅助函数
def filter_word(x):
    word_filter = ['选区', '退换', '差价', '邮费', '补拍']
    for w in word_filter:
        if w in x:
            return True
    return False

def rmpunct(s):
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳/﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃\t﹙+﹛﹝（｛“‘-—_…''')
    return ''.join(filter(lambda x: x not in punct, s))

def extractUrl(url):
    if url.split('.').__len__() < 2:
        return ''
    return url.split('.')[1]

def rmstopwords(s):
    stopwords = set('的和呢了吧')
    return ''.join(filter(lambda x: x not in stopwords, s))

def splitwords(s):
    return ' '.join(jieba.cut(s))

def preprocess(df):
    pass

def tokenize_number(text):
    return re.sub('\d+', '0', text)

def replace_num(texts):
    for i, text in enumerate(texts):
        if text.strip().isdigit():
            texts[i] = '0'
    return texts

In [44]:
#数据预处理
item_info = pd.read_csv('/home/zhuyuhe/mydata/item_profilt_classification/rawdata/item_profile.txt', sep='\t', header=None)
item_info.columns = ['商品标题', '商品url', '商品价格', '商品描述', '品牌', '类目']
#填充缺失值
item_info.商品标题.fillna('', inplace=True)
item_info.商品url.fillna('', inplace=True)
item_info.商品描述.fillna('', inplace = True)
item_info.品牌.fillna('', inplace = True)
item_info.类目.fillna('',inplace = True)
item_info.商品价格.fillna('', inplace = True)
#滤除行
item_info.drop(item_info[item_info.类目.apply(lambda x: x == '')].index, inplace= True)
item_info.drop(item_info[item_info.商品标题.apply(lambda x: x == '')].index, inplace= True)
item_info.drop(item_info[item_info.类目.apply(lambda x: x.split('$').__len__() == 2)].index, inplace=True)

#取特定类目样本
item_list = ['服装配饰', '个护化妆', '鞋']
sub_item = pd.DataFrame()
for i in item_list:
    sub_item = pd.concat([sub_item, item_info[item_info.类目.apply(lambda x: i in x)]])

sub_item_copy = pd.DataFrame(sub_item)
    
#取样本数大于1000的标签进行分类训练    
sub_item.index = range(len(sub_item))
labels = sub_item.类目.value_counts()[:60]
sub_item = sub_item[sub_item.类目.apply(lambda x: x in labels)]

#去符号，抽rul，处理label，去停止词
sub_item.商品标题 = sub_item.商品标题.apply(lambda x: rmpunct(x))
sub_item.商品url = sub_item.商品url.apply(lambda x: extractUrl(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: rmpunct(x))
sub_item.类目 = sub_item.类目.apply(lambda x: '|'.join(x.split('$')[1:]))

sub_item.商品标题 = sub_item.商品标题.apply(lambda x: rmstopwords(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: rmstopwords(x))

sub_item.商品标题 = sub_item.商品标题.apply(lambda x: splitwords(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: splitwords(x))

sub_item['label'] = sub_item.类目.apply(lambda x: '__label__' + x)

sub_item.head()

,商品标题,商品url,商品价格,商品描述,品牌,类目,label
0,咏仕 2016 春季 新品 职业 经典 百搭 纯色 小 西装 工作服 长袖 5A0701,beibei,49,,,女装|西服,__label__女装|西服
1,韩版 时尚 修身 拼色 口袋 休闲 百搭 气质 九分 铅笔 裤,juanpi,55,,,女装|牛仔裤,__label__女装|牛仔裤
2,男子 UA Charged Cotton ™ V 領 T恤,underarmour,249,Under Armour 官方 購物 網站 購買 男子 UA Charged C...,UNDER ARMOUR,女装|T恤,__label__女装|T恤
3,ARMANI JEANS 棉涤 材质 格纹 男士 长袖 衬衫,yintai,619,,阿玛尼,男装|衬衫,__label__男装|衬衫
4,浅休 春 男士 针织 莫代尔 棉 长袖 彩色 条纹 睡衣 套装 1346,yintai,138,,莫代尔,内衣|睡衣,__label__内衣|睡衣


In [45]:
#数据集2数据预处理
item_info2 = pd.read_csv('/home/zhuyuhe/mydata/item_profilt_classification/rawdata/000000_0', sep = '\x01', header = None)
item_info2.columns = ['商品id', '商品url', '品牌', '店铺名称', '商品名称', '类目']

print(len(item_info2))
labels = sub_item.label.unique()

#过滤
item_info2.drop(['商品id', '商品url', '品牌'], axis = 1, inplace = True)
item_info2 = item_info2[item_info2.商品名称.apply(lambda x: not filter_word(x))]
item_info2 = item_info2[item_info2.类目.apply(lambda x: '其他' not in x)]
item_info2 = item_info2[item_info2.类目.apply(lambda x: x != '\\N')]

#选取和之前数据类别有交集的数据
item_info2.类目 = item_info2.类目.apply(lambda x: x.strip('- ').split('-'))
item_info2.类目 = item_info2.类目.apply(lambda x: x[1].split('/')[0]+'|'+x[-1])

# item_info2.品牌 = item_info2.品牌.apply(lambda x: x.replace('\\N', ''))

labels_ = [x.replace('__label__', '') for x in labels]
sub_item2 = item_info2[item_info2.类目.apply(lambda x: x in labels_)]
print(len(sub_item2))

sub_item2.店铺名称 = sub_item2.店铺名称.apply(lambda x: rmpunct(x))
sub_item2.商品名称 = sub_item2.商品名称.apply(lambda x: rmpunct(x))

sub_item2.商品名称 = sub_item2.商品名称.apply(lambda x: rmstopwords(x))

sub_item2.商品名称 = sub_item2.商品名称.apply(lambda x: splitwords(x))

sub_item2['label'] = sub_item2.类目.apply(lambda x: '__label__' + x)
sub_item2.head()

52000
23297


/home/zhuyuhe/.conda/envs/zhuyuheEnv/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/zhuyuhe/.conda/envs/zhuyuheEnv/lib/python3.6/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,店铺名称,商品名称,类目,label
29,ALU ONLINE SHOP,ALU ONLINE SHOP 冰糕 色露 肩 两 穿 质感 好实 穿 连衣裙,女装|连衣裙,__label__女装|连衣裙
38,13c,13C 黑色 舒适 百搭显 瘦 修身 牛奶 丝打 底裤 女裤 BD26,女装|打底裤,__label__女装|打底裤
40,ALU ONLINE SHOP,ALU ONLINE SHOP SS1314 荧光 莓果色 ...,女装|连衣裙,__label__女装|连衣裙
41,ALU ONLINE SHOP,ALU ONLINE SHOP 荧光 莓果色 质感 连衣裙 女 甜美 公主 风收...,女装|连衣裙,__label__女装|连衣裙
43,ALU ONLINE SHOP,ALU ONLINE SHOP 正 红色 皱褶 裹 胸 礼服,女装|连衣裙,__label__女装|连衣裙


In [47]:
sub_item2['concat'] = sub_item2.商品名称  + ' ' + sub_item2.店铺名称
sub_item2 = sub_item2[['concat', 'label']]

sub_item['concat'] = sub_item.商品标题 + ' ' + sub_item.商品描述 + ' ' + sub_item.品牌
sub_item = sub_item[['concat', 'label']]

sub_item_join = pd.concat([sub_item, sub_item2])

sub_item_outer = pd.DataFrame(item_info)
sub_item_outer = sub_item_outer.append(sub_item_copy)
sub_item_outer.drop_duplicates(keep=False, inplace=True)

sub_item_outer.类目 = sub_item_outer.类目.apply(lambda x: x.split('$')[0])
sub_item_outer = sub_item_outer[sub_item_outer.类目.apply(lambda x: '金融' != x and '本地生活' != x)]
sub_item_outer.商品标题 = sub_item_outer.商品标题.apply(lambda x: rmpunct(x))
sub_item_outer.商品描述 = sub_item_outer.商品描述.apply(lambda x: rmpunct(x))

sub_item_outer.商品标题 = sub_item_outer.商品标题.apply(lambda x: rmstopwords(x))
sub_item_outer.商品描述 = sub_item_outer.商品描述.apply(lambda x: rmstopwords(x))

sub_item_outer.商品标题 = sub_item_outer.商品标题.apply(lambda x: splitwords(x))
sub_item_outer.商品描述 = sub_item_outer.商品描述.apply(lambda x: splitwords(x))
sub_item_outer['concat'] = sub_item_outer.商品标题 + sub_item_outer.商品描述 + sub_item_outer.品牌

sub_item_outer['label']  = sub_item_outer.类目.apply(lambda x: '__label__' + x)
sub_item_outer = sub_item_outer[['concat', 'label']]

labels_outer = sub_item_outer.label.unique().tolist()

for l in labels_outer:
    sub_item_join = pd.concat([sub_item_join,sub_item_outer[sub_item_outer.label == l].sample(4000)])


/home/zhuyuhe/.conda/envs/zhuyuheEnv/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
sub_item_join.to_csv('modeldata/sub_item_join.txt', sep = '\t', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')

In [49]:
#划分数据集，80%训练集，20%测试
item_train = pd.DataFrame()
item_valid = pd.DataFrame()
labels = sub_item_join.label.unique()
for l in tqdm(labels):
    sample_train = sub_item_join[sub_item_join.label == l].sample(frac = 0.8)
    sample_valid = sub_item_join[sub_item_join.label == l].drop(sample_train.index)
    item_train = pd.concat([item_train, sample_train])
    item_valid = pd.concat([item_valid, sample_valid])
    
item_train = shuffle(item_train)
item_valid = shuffle(item_valid)

100%|██████████| 76/76 [00:06<00:00, 10.99it/s]


In [50]:
item_train.concat = item_train.concat.apply(lambda x: ' '.join(replace_num(x.split())))
item_valid.concat = item_valid.concat.apply(lambda x: ' '.join(replace_num(x.split())))

In [51]:
#存储，到终端使用fasttext进行训练
item_train.to_csv('modeldata/item_info_fasttext_train_clothes_others.txt', sep = '\t', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')
item_valid.to_csv('modeldata/item_info_fasttext_valid_clothes_others.txt', sep = '\t', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')

In [309]:
#存储，到终端使用fasttext进行训练
item_train[['商品标题', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_train_clothes.txt', sep = '\t', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')
item_valid[['商品标题', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_valid_clothes.txt', sep = '\t', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')

KeyError: "['商品标题' '商品描述' '品牌'] not in index"

In [52]:
item_valid_pre = pd.read_csv('modeldata/item_info_fasttext_valid_clothes_others_result.txt', header = None)

In [53]:
y_test = [y.replace('__label__', '') for y in item_valid.label.values]
y_pre = [y[0].replace('__label__', '') for y in item_valid_pre.values]

In [54]:
print(len(y_test), len(y_pre))

71246 71246


In [55]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pre, digits = 3))

             precision    recall  f1-score   support

  全身护理|口腔护理      0.995     0.993     0.994       611
    全身护理|洗发      0.997     1.000     0.998       327
    内衣|保暖内衣      0.984     0.965     0.974       564
    内衣|女式内裤      0.982     0.994     0.988       882
      内衣|家居      0.945     0.919     0.932       507
    内衣|情趣内衣      0.981     0.970     0.975       263
      内衣|文胸      0.993     0.998     0.995      1513
    内衣|男式内裤      0.992     0.981     0.986       363
      内衣|睡衣      0.983     0.988     0.985       647
      内衣|背心      0.937     0.897     0.916       214
     内衣|连裤袜      0.980     0.965     0.972       200
       医疗保健      0.949     0.934     0.941       800
      女装|T恤      0.968     0.972     0.970      2143
    女装|中老年装      0.976     0.952     0.964       210
     女装|休闲裤      0.942     0.963     0.952      1182
     女装|半身裙      0.980     0.968     0.974      1487
      女装|卫衣      0.968     0.970     0.969       802
     女装|大码装      0.957     0.905     0.931   